In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt 
from project_paths import paths 
from ipywidgets import interact, IntSlider

from lane_detection_medium.inference import DetectionInference
from lane_detection_medium.utils.fs import read_image, read_yolo_labels
from lane_detection_medium.utils.viz import render_bbox
from lane_detection_medium.types.detection_types import ImageDetections
from lane_detection_medium.utils.video_processing import VideoReader

%matplotlib inline

## Trained Models Loading

In [ ]:
EXP_NAME = "train-2023-07-05"

CHECKPOINT_DPATH = paths.yolo_dpath / "LaneMarkingsDetection" / EXP_NAME / "weights"
MODEL_FPATH = CHECKPOINT_DPATH / "best.pt"

inference_wobg = DetectionInference.from_file(
    str(MODEL_FPATH), 
    device="cuda:0", 
    img_size=(640, 640)
)

In [ ]:
EXP_NAME = "train-2023-07-10"

CHECKPOINT_DPATH = paths.yolo_dpath / "LaneMarkingsDetection" / EXP_NAME / "weights"
MODEL_FPATH = CHECKPOINT_DPATH / "best.pt"

inference_wbg = DetectionInference.from_file(
    str(MODEL_FPATH), 
    device="cuda:0", 
    img_size=(640, 640)
)

## Vizualization 

In [ ]:
data_dpath = paths.yolo_dpath / "data" / "2023_07_10" / "test"
img_dpath = data_dpath / "images"
lbl_dpath = data_dpath / "labels"

img_fpaths = sorted(list((data_dpath / "images").glob("*.PNG")))
txt_fpaths = sorted(list((data_dpath / "labels").glob("*.txt")))

len(img_fpaths), len(txt_fpaths)

In [ ]:
color_map = { 
  "solid_white": (255, 0, 0), 
  "break_white": (0, 0, 255), 
  "zebra": (255, 255, 0)
}

@interact 
def show_inference(index=IntSlider(val=0, min=0, max=len(txt_fpaths) - 1)):
    txt_fpath = txt_fpaths[index]
    img_fpath = img_dpath / f"{txt_fpath.stem}.PNG"

    test_image = read_image(str(img_fpath))
    gt_canva = test_image.copy()
    wobg_canva = test_image.copy() 
    wbg_canva = test_image.copy()

    gt_np = read_yolo_labels(txt_fpath)
    gt_dets = ImageDetections.from_yolo_labels(gt_np, *test_image.shape[:2])

    wobg_detections = inference_wobg.detect([test_image], conf=0.25)[0]
    wbg_detections = inference_wbg.detect([test_image], conf=0.25)[0]

    fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(15, 25))

    # --- Ground Truth Rendering --- #
    for gt_det in gt_dets:
        label_name = inference_wobg.names_map[gt_det.label_id]
        render_bbox(
            gt_canva, 
            gt_det.bbox, 
            label=label_name, 
            color=color_map[label_name], 
            font_color=(255, 255, 255),
            line_thickness=4, 
            font_size=1.5
        )
    ax[0].imshow(gt_canva)
    ax[0].set_title("Ground Truth")

    # --- Inference w/o background frames --- # 
    for det in wobg_detections:
        label_name = f"{det.label_name}: {det.conf:.2f}"
        render_bbox(
            wobg_canva, det.bbox, label=label_name, color=color_map[det.label_name], font_color=(255, 255, 255), 
            line_thickness=4, 
            font_size=1.5
        )
    ax[1].imshow(wobg_canva)
    ax[1].set_title("Inference w/o background")

    # --- Inference w/ background frames --- # 
    for det in wbg_detections:
        label_name = f"{det.label_name}: {det.conf:.2f}"
        render_bbox(
            wbg_canva, det.bbox, label=label_name, color=color_map[det.label_name], font_color=(255, 255, 255), 
            line_thickness=4, 
            font_size=1.5
        )
    ax[2].imshow(wbg_canva)
    ax[2].set_title("Inference w/ background")

    plt.show()

## Background Frames

In [ ]:
video_fname = "no_lines.mp4"
video_fpath = paths.data / "videos" / video_fname

with VideoReader(video_fpath, verbose=True) as reader: 
    frames = list(reader.get_frames()) 

In [ ]:
color_map = { 
  "solid_white": (255, 0, 0), 
  "break_white": (0, 0, 255), 
  "zebra": (255, 255, 0)
}

@interact 
def show_inference(index=IntSlider(val=0, min=0, max=len(frames) - 1)):
    test_image = frames[index]
    wobg_canva = test_image.copy() 
    wbg_canva = test_image.copy()

    wobg_detections = inference_wobg.detect([test_image], conf=0.25)[0]
    wbg_detections = inference_wbg.detect([test_image], conf=0.25)[0]

    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 18))


    # --- Inference w/o background frames --- # 
    for det in wobg_detections:
        label_name = f"{det.label_name}: {det.conf:.2f}"
        render_bbox(
            wobg_canva, det.bbox, label=label_name, color=color_map[det.label_name], font_color=(255, 255, 255), line_thickness=4, 
            font_size=1.5
        )
    ax[0].imshow(wobg_canva)
    ax[0].set_title("Inference w/o background")

    # --- Inference w/ background frames --- # 
    for det in wbg_detections:
        label_name = f"{det.label_name}: {det.conf:.2f}"
        render_bbox(
            wbg_canva, det.bbox, label=label_name, color=color_map[det.label_name], font_color=(255, 255, 255), line_thickness=4, 
            font_size=1.5
        )
    ax[1].imshow(wbg_canva)
    ax[1].set_title("Inference w/ background")

    plt.show()